In [41]:
import pandas as pd
import plotly.express as px
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV
from haversine import haversine
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_csv('The_full_info_cian.csv')

#df = df[df.Price < 50 * 10**6]
# df = df[df.Area < 500]
df = df.drop(columns = ['Unnamed: 0', 'id',
                        'date'])

df = df[(df.type != 'aerocreteBlock')&(df.type != 'foamConёёёёёёёёёёёёёёёёёёёёёиcreteBlock')&(df.type != 'wood')&(df.type != 'boards')&(df.type != 'old')]

In [29]:
px.histogram(df[df.Price < 100*10**6], 'Price', marginal='box', nbins=100, color_discrete_sequence=['#3c43a5'], template = 'simple_white',
            title='Распределение цен в датасете', labels = {'count':'количество'})

In [ ]:
X = df.drop(columns=['Price'])  # Признаки 
y = df['Price']  # Таргет
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42
                 )

# CATBOOST

In [ ]:
#Обучим модель, используя CatBoost
model0 = CatBoostRegressor(iterations=5000,  # количество деревьев
                          learning_rate=0.05,  # скорость обучения
                          depth=9,  # глубина деревьев
                          random_state=42,  # для воспроизводимости результатов
                          verbose=250)  # подробность вывода информации о процессе обучения
# Обучение
model0.fit(X_train, y_train, cat_features=[0, 4, 15, 16, 17])  # Указываем индексы категориальных признаков

# Прогнозa
y_pred0 = model0.predict(X_test)


mse1 = mean_squared_error(y_test, y_pred0)
rmse1 = np.sqrt(mean_squared_error(y_test, y_pred0))
mae1 = mean_absolute_error(y_test, y_pred0)
mape1 = np.mean(np.abs((y_test - y_pred0) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting:', 
    '\nMSE: ', mse1,
     '\nRMSE: ', rmse1, 
     '\nMAE: ', mae1, 
     '\nMAPE: ', mape1,
     '\n\nRMSE in millions: ', (rmse1/1000000).round(3), 'millions')

In [15]:
# Подготовим к OneHotEncoding
X_len = len(X_train)
datas = pd.concat([X_train, X_test], axis = 0)
datas = pd.get_dummies(datas)
X_train = datas.iloc[:X_len, :]
X_test = datas.iloc[X_len:, :]

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# LIGHT GBM

In [223]:
import lightgbm as lgb
params = {
    'objective': 'regression',  # Функция потерь для регрессии
    'metric': 'rmse',  # Метрика качества
    'learning_rate': 0.125,  # Скорость обучения
    'max_depth': 10,  # Максимальная глубина деревьев
    'seed': 42  # Для воспроизводимости результатов
}
num_rounds = 10000  # Количество итераций (деревьев)
model1 = lgb.train(params, train_data, num_rounds, valid_sets=[test_data])

# Прогноз
y_pred1 = model1.predict(X_test, num_iteration = num_rounds)

mse = mean_squared_error(y_test, y_pred1)
rmse = np.sqrt(mean_squared_error(y_test, y_pred1))
mae = mean_absolute_error(y_test, y_pred1)
mape = np.mean(np.abs((y_test - y_pred1) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting с LightGBM:', 
    '\nMSE: ', mse,
    '\nRMSE: ', rmse, 
    '\nMAE: ', mae, 
    '\nMAPE: ', mape,
    '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001989 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3275
[LightGBM] [Info] Number of data points in the train set: 29568, number of used features: 335
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 20001949.895123
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

# XGBoost

In [224]:
import xgboost as xgb
params2 = {
    'objective': 'reg:squarederror',  # Функция потерь для регрессии
    'learning_rate': 0.15,  # Скорость обучения
    'max_depth': 10,  # Максимальная глубина деревьев
    'seed': 42  # Для воспроизводимости результатов
}
num_rounds = 10000  # Количество итераций (деревьев)
model2 = xgb.train(params2, dtrain, num_rounds)

y_pred2 = model2.predict(dtest)

mse2 = mean_squared_error(y_test, y_pred2)
rmse2 = np.sqrt(mean_squared_error(y_test, y_pred2))
mae2 = mean_absolute_error(y_test, y_pred2)
mape2 = np.mean(np.abs((y_test - y_pred2) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting с XGBoost:', 
    '\nMSE: ', mse2,
    '\nRMSE: ', rmse2, 
    '\nMAE: ', mae2, 
    '\nMAPE: ', mape2,
    '\n\nRMSE in millions: ', (rmse2/1000000).round(3), 'millions')


РЕЗУЛЬТАТЫ Gradient Boosting с XGBoost: 
MSE:  9247488352675.434 
RMSE:  3040968.324839217 
MAE:  1912465.6313582251 
MAPE:  10.909494011683005 

RMSE in millions:  3.041 millions


In [225]:
model3 = RandomForestRegressor(n_estimators=5000, max_depth=7, random_state=42)
model3.fit(X_train, y_train)

y_pred3 = model3.predict(X_test)

mse3 = mean_squared_error(y_test, y_pred3)
rmse3 = np.sqrt(mse)
mae3 = mean_absolute_error(y_test, y_pred3)
mape3 = np.mean(np.abs((y_test - y_pred3) / y_test)) * 100

print('РЕЗУЛЬТАТЫ Random Forest:', 
    '\nMSE: ', mse3,
     '\nRMSE: ', rmse3, 
     '\nMAE: ', mae3, 
     '\nMAPE: ', mape3,
     '\n\nRMSE in millions: ', (rmse3/1000000).round(3), 'millions')

РЕЗУЛЬТАТЫ Random Forest: 
MSE:  16187242539719.822 
RMSE:  3035884.5719371364 
MAE:  2799808.1187046585 
MAPE:  15.674081577802939 

RMSE in millions:  3.036 millions


In [234]:
mse_best = 10 ** 1000
for w0 in range(1, 30):
    for w1 in range(0, 20):
        for w2 in range(0, 20):
            for w3 in range(0, 20):
                y_pred = (w0 * y_pred0 + w1 * y_pred1 + w2 * y_pred2 + w3 * y_pred3)/(w1 + w2 + w0 + w3)
                mse = mean_squared_error(y_test, y_pred)
                rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                mae = mean_absolute_error(y_test, y_pred)
                mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
                if mse < mse_best:
                    mse_best = mse
                    print('Веса: ', w0, w1, w2, w3, f'  RMSE: {rmse}')
                    w0_best = w0
                    w1_best = w1
                    w2_best = w2
                    w3_best = w3
print('w0, w1, w2, w3 = ', f'{w0_best}, {w1_best}, {w2_best}, {w3_best}')

Веса:  1 0 0 0   RMSE: 2980274.560017969
Веса:  1 0 1 0   RMSE: 2918320.620474906
Веса:  1 1 0 0   RMSE: 2917261.4290811634
Веса:  1 1 1 0   RMSE: 2899150.140097184
Веса:  2 1 1 0   RMSE: 2895812.0054840995
Веса:  3 2 2 0   RMSE: 2895290.0550798015
Веса:  4 3 3 1   RMSE: 2893979.2182491953
Веса:  4 4 3 1   RMSE: 2893350.3501316872
Веса:  5 4 3 1   RMSE: 2892421.410115946
Веса:  6 4 4 1   RMSE: 2892036.4274787433
Веса:  6 5 4 1   RMSE: 2891740.844399027
Веса:  7 5 4 1   RMSE: 2891642.227250587
Веса:  8 6 5 1   RMSE: 2891487.1720601637
Веса:  22 16 14 3   RMSE: 2891482.8771786364
Веса:  23 17 15 3   RMSE: 2891478.6795923845
Веса:  24 17 15 3   RMSE: 2891474.548762131
w0, w1, w2, w3 =  24, 17, 15, 3


In [238]:
w0, w1, w2, w3 =  1, 1, 1, 0
#w0, w1, w2, w3 =  24, 17, 15, 3


y_pred = (w0 * y_pred0 + w1 * y_pred1 + w2 * y_pred2 )/(w1 + w2 + w0 )
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ ВСЕ МОДЕЛИ:', 
    '\nMSE: ', mse,
    '\nRMSE: ', rmse, 
    '\nMAE: ', mae, 
    '\nMAPE: ', mape,
    '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')


РЕЗУЛЬТАТЫ ВСЕ МОДЕЛИ: 
MSE:  8405071534825.521 
RMSE:  2899150.140097184 
MAE:  1826760.4267799326 
MAPE:  10.435266049945383 

RMSE in millions:  2.899 millions
